In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
! pip install adam-mini
! pip install -U wandb
! pip install evaluate
! pip install peft==0.12.0
! pip install -U bitsandbytes

/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv
/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
/kaggle/input/w2v-nlp/testData.tsv
/kaggle/input/w2v-nlp/labeledTrainData.tsv
/kaggle/input/w2v-nlp/unlabeledTrainData.tsv
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 41.2 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.18.3
    Uninstalling wandb-0.18.3:
      Successfully uninstalled wandb-0.18.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.3 MB/s eta 0:00:00


In [2]:
import os
import sys
import logging
from time import sleep

import datasets
import evaluate

import pandas as pd
import numpy as np

import wandb
from adam_mini import Adam_mini
from torch.onnx.utils import setup_onnx_logging

from transformers import BitsAndBytesConfig

from transformers import AutoModelForSequenceClassification, DebertaV2Tokenizer, DataCollatorWithPadding
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.model_selection import train_test_split

train = pd.read_csv("/kaggle/input/w2v-nlp/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("/kaggle/input/w2v-nlp/testData.tsv", header=0, delimiter="\t", quoting=3)

# class customTrainer(Trainer):
#     def create_optimizer(self):
#         optimizer = Adam_mini(
#             named_parameters=self.model.named_parameters(),
#             lr=self.args.learning_rate,
#             betas=self.model.args.betas,
#             weight_decay=self.model.config.weight_decay,
#             dim=self.model.config.dim,
#             n_heads=self.model.config.n_heads,
#             n_kv_heads=self.model.config.n_kv_heads,
#             eps=self.model.config.eps,
#         )
#         self.optimizer = optimizer



if __name__ == '__main__':
    os.environ["WANDB_PROJECT"] = "deberta_lora_xlarge"
    os.environ['WANDB_API_KEY'] = "a464ce6c3b972e3e7090ac20839b9a1daac1b608"
    wandb.init()
    train, val = train_test_split(train, test_size=.2)

    train_dict = {'label': train["sentiment"], 'text': train['review']}
    val_dict = {'label': val["sentiment"], 'text': val['review']}
    test_dict = {"text": test['review']}

    train_dataset = datasets.Dataset.from_dict(train_dict)
    val_dataset = datasets.Dataset.from_dict(val_dict)
    test_dataset = datasets.Dataset.from_dict(test_dict)

    # batch_size = 32

    model_id = "microsoft/deberta-v2-xlarge"

    tokenizer = DebertaV2Tokenizer.from_pretrained(model_id)


    def preprocess_function(examples):
        return tokenizer(examples['text'], truncation=True)
        # 先测试不加入max_length的版本

    tokenized_train = train_dataset.map(preprocess_function, batched=True)
    tokenized_val = val_dataset.map(preprocess_function, batched=True)
    tokenized_test = test_dataset.map(preprocess_function, batched=True)

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    model = AutoModelForSequenceClassification.from_pretrained(
        model_id,
#         quantization_config=BitsAndBytesConfig(load_in_4bit=True),
        # load_in_8bit = True,
        # device_map="auto",
        # load_in_8bit=True
    )

    # Define LoRA Config
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        # target_modules=['q_proj', 'v_proj'],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.SEQ_CLS
    )

    # prepare int-8 model for training
    # model = prepare_model_for_int8_training(model)

    # add LoRA adaptor
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    metric = evaluate.load("accuracy")
    optimizer = Adam_mini(
        named_parameters=model.named_parameters(),
        lr=2e-5,
        betas=(0.9, 0.999),
        weight_decay=0.01,
        # 其他参数
    )

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
#         wandb.log({
#             "accuracy":ans,
#         })
        return metric.compute(predictions=predictions, references=labels)



    training_args = TrainingArguments(
        output_dir='./checkpoint',  # output directory
        report_to="wandb",
        num_train_epochs=3,  # total number of training epochs
        per_device_train_batch_size=2,  # batch size per device during training
        per_device_eval_batch_size=4,  # batch size for evaluation
        warmup_steps=500,  # number of warmup steps for learning rate scheduler
        weight_decay=0.01,  # strength of weight decay
        logging_dir='./logs',  # directory for storing logs
        logging_steps=100,
        save_strategy="no",
        evaluation_strategy="steps",
#         gradient_accumulation_steps=8, 
    )

    trainer = Trainer(
        model=model,  # the instantiated 🤗 Transformers model to be trained
        args=training_args,  # training arguments, defined above
        train_dataset=tokenized_train,  # training dataset
        eval_dataset=tokenized_val,  # evaluation dataset
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        optimizers=(optimizer, None),
    )

    trainer.train()

    prediction_outputs = trainer.predict(tokenized_test)
    test_pred = np.argmax(prediction_outputs[0], axis=-1).flatten()


    result_output = pd.DataFrame(data={"id": test["id"], "sentiment": test_pred})
    if not os.path.exists("./result"):
        os.mkdir("./result")
    result_output.to_csv("./result/deberta_lora.csv", index=False, quoting=3)
    wandb.finish()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: skyfurynowonline (skyfurynowonline-yunnan-university). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.5
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241030_124146-815hwtsx
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run crawly-wizard-9
wandb: ⭐️ View project at https://wandb.ai/skyfurynowonline-yunnan-university/deberta_lora_xlarge
wandb: 🚀 View run at https://wandb.ai/skyfurynowonline-yunnan-university/deberta_lora_xlarge/runs/815hwtsx


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v2-xlarge and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 2,362,370 || all params: 889,319,428 || trainable%: 0.2656


Adam-mini found the param block with name: base_model.model.deberta.encoder.layer.0.attention.self.query_proj.lora_A.default.weight torch.Size([16, 1536])
Adam-mini found the param block with name: base_model.model.deberta.encoder.layer.0.attention.self.query_proj.lora_B.default.weight torch.Size([1536, 16])
Adam-mini found the param block with name: base_model.model.deberta.encoder.layer.0.attention.self.value_proj.lora_A.default.weight torch.Size([16, 1536])
Adam-mini found the param block with name: base_model.model.deberta.encoder.layer.0.attention.self.value_proj.lora_B.default.weight torch.Size([1536, 16])
Adam-mini found the param block with name: base_model.model.deberta.encoder.layer.1.attention.self.query_proj.lora_A.default.weight torch.Size([16, 1536])
Adam-mini found the param block with name: base_model.model.deberta.encoder.layer.1.attention.self.query_proj.lora_B.default.weight torch.Size([1536, 16])
Adam-mini found the param block with name: base_model.model.deberta.en

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 43.12 MiB is free. Process 2925 has 15.84 GiB memory in use. Of the allocated memory 15.24 GiB is allocated by PyTorch, and 322.58 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)